In [2]:
import os, glob, time, re, json, random
import cv2
import matplotlib.pyplot as plt
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

class PreprocessDownstream:
    def __init__(self, videoPathL, outputPath, videoLength=2.4, labelPath=None, classL=None, divide=5, posThreshold=0.75, negThreshold=0.0):
        self.videoPathL  = videoPathL
        self.outputPath  = outputPath
        self.videoLength = videoLength
        self.labelD = self.getLabel(labelPath)
        self.classL = classL + ["others"]
        flags = [ round(videoLength/divide*i,2)  for i in range(divide+1) ] # e.g. videoLength=2.4, divide=5, flag=[0.0, 0.48, 0.96, 1.44, 1.92, 2.4]
        self.shifts = shifts = [0]+[round(flags[i]+(flags[i+1]-flags[i])*random.Random(i+7).random(),2) for i in range(divide)]
        print(f"self.shifts={shifts}")
        self.divide = divide
        self.posThreshold = posThreshold
        self.negThreshold = negThreshold
        
    def getOverallInfo(self, show=False):
        for i,videoPath in enumerate(self.videoPathL):
            frames, fps, height, width = self.getVideoInfo(videoPath)
            print(f"{i}, {videoPath.split('/')[-1]}, frames={frames}, fps={fps}, height={height}, width={width}")
            self.show1Frame(videoPath, self.rotateL[i]) if show else None
    
    def getVideoInfo(self, videoPath):
        cap    = cv2.VideoCapture(videoPath)
        frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        fps    = cap.get(cv2.CAP_PROP_FPS)
        height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        cap.release()
        return int(frames), round(fps), int(height), int(width)
    
    def show1Frame(self, videoPath):
        cap = cv2.VideoCapture(videoPath)
        success, img = cap.read()
        if success:
            plt.imshow(img[:,:,::-1])
            plt.show()
        cap.release()
        
    def getLabel(self, labelPath):
        labelD = {}
        for line in open(labelPath,"r").readlines():
            line = line.replace("\n","").replace(" ","")
            if "[" in line:
                videoName = line[1:-1]
                labelD[videoName] = []
            else:
                for cid,timeSlot in enumerate( line.split(",") ):
                    if timeSlot in ['','-'*23]:
                        continue
                    start, end = timeSlot.split("-")
                    (sh,sm,ss), (eh,em,es) = start.split(":"), end.split(":")
                    start, end = int(sh)*3600+int(sm)*60+float(ss), int(eh)*3600+int(em)*60+float(es)
                    assert 2.5<=end-start<=4, timeSlot                    
                    labelD[videoName].append( (cid,start,end) )
        print(f"self.labelD={labelD}")
        return labelD
        
    def clip(self):
        for i,cl in enumerate(self.classL):
            os.makedirs(f"{self.outputPath}/{i}_{cl}", exist_ok=True)
        #
        for i,videoPath in enumerate(self.videoPathL):
            frames, fps, _, _ = self.getVideoInfo(videoPath) 
            videoTime = round(frames/fps) # ~297 sec
            videoName = videoPath.split("/")[-1]
            print(f"\r{i+1}/{len(self.videoPathL)}, {videoName}", end="")
            #
            clip = mp.VideoFileClip(videoPath)
            clip = clip.resize( (568,320) ) # modify aspect ratio (w,h)    
            gtL  = self.labelD[videoName]
            cutL = [ (round(self.videoLength*j+shift,2),round(self.videoLength*(j+1)+shift,2)) for j in range(int(videoTime/self.videoLength)-1) for shift in self.shifts ]
            for start,end in cutL:
                insertL = sorted( gtL[:]+[(0,start,end)], key=lambda tup:tup[1])
                idx = insertL.index( (0,start,end) )
                lastCat, lastStart, lastEnd = insertL[idx-1] if idx>0 else (None,-999,-999)
                nextCat, nextStart, nextEnd = insertL[idx+1] if idx<len(insertL)-1 else (None,999,999)
                nextOverlap = (end-nextStart)/self.videoLength
                lastOverlap = (lastEnd-start)/self.videoLength
                if nextOverlap>self.posThreshold:
                    cat = nextCat
                elif lastOverlap>self.posThreshold:
                    cat = lastCat
                elif nextOverlap>=self.negThreshold or lastOverlap>=self.negThreshold:
                    cat = -1
                else:
                    cat = len(self.classL)-1 if random.random()<1/(self.divide) else -1
                #print( (lastCat,lastStart,lastEnd), (nextCat, nextStart, nextEnd), (start,end), cat )
                try:
                    assert cat!=-1
                    subclip = clip.subclip(start,end)
                    savePath = f"{self.outputPath}/{cat}_{self.classL[cat]}/" + videoName.replace('.mp4',f'_{cat}_{start}.mp4')
                    subclip.write_videofile(savePath, verbose=False, logger=None)
                except Exception as e:
                    open(f"./{self.outputPath}/log.txt","a").write(e) if cat!=-1 else None
                            
    def getClassDistribution(self):
        self.classCountL = [0]*len(self.classL)
        for newVideoPath in glob.glob(f"{self.outputPath}/*/*.mp4"):
            cid, _ = newVideoPath.split("/")[-2].split("_")
            self.classCountL[int(cid)]+=1
        print(f"self.classCountL={self.classCountL}")
        
    def check(self):
        newVideoPathL = sorted(glob.glob(f"{self.outputPath}/*/*.mp4"))
        for i,newVideoPath in enumerate(newVideoPathL):
            print(f"\r{i+1}/{len(newVideoPathL)}", end="")
            cap = cv2.VideoCapture(newVideoPath)
            success, img = cap.read()
            assert bool(success), newVideoPath
            frames, fps, height, width = self.getVideoInfo(newVideoPath)
            assert abs(frames-1)<=fps*self.videoLength, (frames,newVideoPath)
            cap.release()
        print("\ncheck video and frames complete")
    
    def generate_csv(self):
        newVideoPathL = glob.glob(f"{self.outputPath}/*/*.mp4")
        random.shuffle(newVideoPathL)
        cut = int(len(newVideoPathL)*0.8)
        for dset,(start,end) in zip(["train","val","test"], [(None,cut),(cut,None),(cut,None)]):
            with open(f"{self.outputPath}/{dset}.csv","w") as f:
                for newVideoPath in newVideoPathL[start:end]:
                    f.write(f"{os.path.abspath(newVideoPath)} {newVideoPath.split('/')[-2].split('_')[0]}\n")

In [4]:
videoPathL = sorted(glob.glob("/home/jovyan/data-vol-2/HAR/C10/20220810/*.mp4"))[:10]
obj = PreprocessDownstream( videoPathL, "../_data/downstream_0810_10_v2", labelPath="../_data/downstream_0810_10_v2/label.txt", classL=["scan","tear","close"], divide=5)
obj.getOverallInfo()
#obj.clip()
obj.getClassDistribution()
obj.check()
obj.generate_csv()

self.labelD={'video_20220810080429.mp4': [], 'video_20220810080929.mp4': [], 'video_20220810081429.mp4': [], 'video_20220810081929.mp4': [], 'video_20220810082429.mp4': [(0, 251.0, 253.5), (1, 265.5, 268.0), (2, 293.5, 296.5)], 'video_20220810082929.mp4': [(0, 267.75, 270.25), (1, 281.0, 284.0)], 'video_20220810083429.mp4': [(2, 6.0, 9.0), (0, 49.75, 52.25), (1, 64.0, 67.0), (2, 85.0, 88.0), (0, 108.75, 111.25), (1, 118.5, 121.5), (2, 143.0, 145.5), (0, 160.25, 162.75), (1, 198.0, 200.5), (2, 226.5, 229.5), (0, 273.75, 276.25), (1, 287.5, 290.0)], 'video_20220810083930.mp4': [(2, 13.0, 15.5), (0, 34.0, 36.5), (1, 44.0, 46.5), (2, 60.5, 63.5), (0, 106.0, 108.5), (1, 123.0, 125.5), (2, 145.5, 148.5), (0, 220.5, 223.0), (1, 237.0, 239.5), (2, 254.0, 257.0), (0, 276.5, 279.0), (1, 288.5, 292.0)], 'video_20220810084430.mp4': [(2, 12.5, 15.5), (0, 46.75, 49.25), (1, 56.5, 59.0), (2, 81.0, 84.0), (0, 98.0, 100.5), (1, 114.0, 116.5), (2, 126.5, 129.0), (0, 144.75, 147.25), (1, 184.0, 187.0), (